## Shashwat Jha
#### MIDAS Summer Internship Task (Task3 - NLP)

In [1]:
import pandas as pd
import re 
import nltk 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score, precision_score, f1_score

In [2]:
#Reading the csv file using pandas
df = pd.read_csv("flipkart.csv")

In [3]:
#To view the frist five rows of the dataframe
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
#Outputs the number of rows and columns in the daqtaframe
df.shape

(20000, 15)

In [5]:
#Name of all the columns in our dataframe
df.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

In [6]:
#Ouputs the statistical features of only numerical columns
df.describe()

,retail_price,discounted_price
count,19922.000000,19922.000000
mean,2979.206104,1973.401767
std,9009.639341,7333.586040
min,35.000000,35.000000
25%,666.000000,350.000000
50%,1040.000000,550.000000
75%,1999.000000,999.000000
max,571230.000000,571230.000000


In [7]:
#Used to ouput total number of non-null values in each row, data type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In this task we will take the "description" columns as our main feature.<br>
We can see that we have two null values in description column, in the codeblock below we are trying to get the index of these two instances.

In [8]:
df[df['description'].isnull()].index

Int64Index([553, 17299], dtype='int64')

In [9]:
#Printing out the description of the index 553 , 17299
print(df['description'][553])
print(df['description'][17299])

nan
nan


If we have decided to take description column as the main feature, we can drop the above two mentioned instances from our data frame as it doesn't have any description and thus it can't be used for either training or testing our model. 

In [10]:
df.drop([553, 17299] , inplace = True)

In [11]:
# We have dropped two rows
df.shape

(19998, 15)

In [12]:
#Let's print out one random description to see what does our features look like 
print(df['description'][37])

Specifications of Pick Pocket Embroidered Women's Waistcoat General Details Pattern Embroidered Occasion Casual Ideal For Women's Waistcoat Details Lining Polyester Fabric Bengal Silk Additional Details Style Code TL-012 Fabric Care First Time Dry Clean


We see that there are many characters in the description which are not useful for us. Thus the next step that we adopt of cleaning the strings of the desription column. We can clean these strings with the help of *regex* library inbuilt in python.

In [13]:
def preprocessing_string(str_arg):
    '''
    input: str_arg --> Takes string to clean
    output: cleaned_str --> Gives back cleaned string
    This fuction cleans the text in the mentioned ways as comments after the line.This has been copied from some other kernel.

    '''
    cleaned_str=re.sub('[^a-z\s]+',' ',str_arg,flags=re.IGNORECASE) #every char except alphabets is replaced
    cleaned_str=re.sub('(\s+)',' ',cleaned_str) #multiple spaces are replaced by single space
    cleaned_str=cleaned_str.lower() #converting the cleaned string to lower case
    
    return cleaned_str # Returning the preprocessed string in tokenized form

In [14]:
'''
    Lets see how our string look after we appply the "preprocessing_string" function on it.
'''
print(df["description"][0])
print('\n')
print(preprocessing_string(df["description"][0]))

Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts


key features of alisha solid women s cycling shorts cotton lycra navy red navy specifications of alisha solid women s cycling shorts shorts details number of contents in sales package pack of fabric cotton lycra type cycling shorts general details pattern solid ideal for women s fabric care gentle machine wash in lukewarm water do not bleach additional details style code altht p in the box shorts


As mentioned in the task, the main goal is to find the category of product. The "product_category_tree" column in dataframe is our target.<br>

In [15]:
#lets look at the product_category_tree column
df['product_category_tree']

0        ["Clothing >> Women's Clothing >> Lingerie, Sl...
1        ["Furniture >> Living Room Furniture >> Sofa B...
2        ["Footwear >> Women's Footwear >> Ballerinas >...
3        ["Clothing >> Women's Clothing >> Lingerie, Sl...
4        ["Pet Supplies >> Grooming >> Skin & Coat Care...
                               ...                        
19995    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
19996    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
19997    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
19998    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
19999    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
Name: product_category_tree, Length: 19998, dtype: object

We can see that there are numerous categories in the dataframe, also we can notice that in each category there are some sub categories, as per the task requirement we only need focus on the "main" or so called "parent" categories.<br>
For this purpose we need to process these categories in order to extract the "main" category of the product.

In [16]:
# Category processing. (Check data to understand)
df['product_category_tree'] = df['product_category_tree'].apply(lambda x : x.split('>>')[0][2:].strip())

In [17]:
#lets look at some of the main categories extracted by the help of the lambda, split and strip.
print(df['product_category_tree'][50])
print(df["product_category_tree"][0])

Vishudh Printed Women's Straight Kurta"]
Clothing


In [18]:
#Total number of uniques "parent"
df['product_category_tree'].nunique()

266

In [19]:
#Printing out all the unique categories
df['product_category_tree'].unique()

array(['Clothing', 'Furniture', 'Footwear', 'Pet Supplies',
       'Eternal Gandhi Super Series Crystal Paper Weight..."]',
       'Pens & Stationery', 'Sports & Fitness',
       'Beauty and Personal Care',
       'Bengal Blooms Rose Artificial Plant  with Pot (3..."]',
       'Bags, Wallets & Belts', 'Home Decor & Festive Needs',
       'Automotive', 'Tools & Hardware',
       'Vishudh Printed Women\'s Straight Kurta"]',
       'Vishudh Printed Women\'s Anarkali Kurta"]',
       'BuildTrack PIR Wireless Motion Sensor - One Swit..."]',
       'Skayvon SUMMERSIBLE SINGLE PHASE PUMP CONTROLLER..."]',
       'MASARA Solid Women\'s Straight Kurta"]',
       'Skayvon SUBMERSIBBLE THREE PHASE PUMP CONTROLLER..."]',
       'Behringer Xenyx 502 Analog Sound Mixer"]',
       'Noor Embroidered Women\'s Straight Kurta"]',
       'Libas Printed Women\'s A-line Kurta"]',
       'Libas Printed Women\'s Anarkali Kurta"]', 'Home Furnishing',
       'Baby Care', 'Mobiles & Accessories', 'Food & Nutriti

As we can notice there are many examples where the product doesn't have a definite category.<br>
For example instead of categorising the product under clothing, its rather filled the product detail<br>
See the code cell below.

In [20]:
print(df['product_category_tree'][50])

Vishudh Printed Women's Straight Kurta"]


In order to prevent such intsnaces where the product category is not define we will chose only those categories which which have a definite property such as clothing, footwear, camera and acessories etc

In [21]:
# Taking only top categories
top_cat = list(df.groupby('product_category_tree').count().sort_values(by='uniq_id',ascending=False).head(28).index)

In [22]:
print(top_cat)

['Clothing', 'Jewellery', 'Footwear', 'Mobiles & Accessories', 'Automotive', 'Home Decor & Festive Needs', 'Beauty and Personal Care', 'Home Furnishing', 'Kitchen & Dining', 'Computers', 'Watches', 'Baby Care', 'Tools & Hardware', 'Toys & School Supplies', 'Pens & Stationery', 'Bags, Wallets & Belts', 'Furniture', 'Sports & Fitness', 'Cameras & Accessories', 'Home Improvement', 'Health & Personal Care Appliances', 'Sunglasses', 'Gaming', 'Pet Supplies', 'Home & Kitchen', 'Home Entertainment', 'eBooks', 'Eyewear']


In [23]:
# Selecting only relevant columns
processed_df = df[df['product_category_tree'].isin(top_cat)][['product_category_tree','description']]

In [24]:
processed_df

,product_category_tree,description
0,Clothing,Key Features of Alisha Solid Women's Cycling S...
1,Furniture,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,Footwear,Key Features of AW Bellies Sandals Wedges Heel...
3,Clothing,Key Features of Alisha Solid Women's Cycling S...
4,Pet Supplies,Specifications of Sicons All Purpose Arnica Do...
...,...,...
19995,Baby Care,Buy WallDesign Small Vinyl Sticker for Rs.730 ...
19996,Baby Care,Buy Wallmantra Large Vinyl Stickers Sticker fo...
19997,Baby Care,Buy Elite Collection Medium Acrylic Sticker fo...
19998,Baby Care,Buy Elite Collection Medium Acrylic Sticker fo...


In [25]:
# Cleaning strings
processed_df['description'] = processed_df['description'].astype('str').apply(preprocessing_string)

In [26]:
processed_df

,product_category_tree,description
0,Clothing,key features of alisha solid women s cycling s...
1,Furniture,fabhomedecor fabric double sofa bed finish col...
2,Footwear,key features of aw bellies sandals wedges heel...
3,Clothing,key features of alisha solid women s cycling s...
4,Pet Supplies,specifications of sicons all purpose arnica do...
...,...,...
19995,Baby Care,buy walldesign small vinyl sticker for rs onli...
19996,Baby Care,buy wallmantra large vinyl stickers sticker fo...
19997,Baby Care,buy elite collection medium acrylic sticker fo...
19998,Baby Care,buy elite collection medium acrylic sticker fo...


In [27]:
# Creating a list of categories for later use
cat_list = list(processed_df['product_category_tree'].unique())

In [28]:
# Printing the list of top categories
print(cat_list)

['Clothing', 'Furniture', 'Footwear', 'Pet Supplies', 'Pens & Stationery', 'Sports & Fitness', 'Beauty and Personal Care', 'Bags, Wallets & Belts', 'Home Decor & Festive Needs', 'Automotive', 'Tools & Hardware', 'Home Furnishing', 'Baby Care', 'Mobiles & Accessories', 'Watches', 'Toys & School Supplies', 'Jewellery', 'Kitchen & Dining', 'Home & Kitchen', 'Computers', 'Cameras & Accessories', 'Health & Personal Care Appliances', 'Gaming', 'Home Improvement', 'Sunglasses', 'Home Entertainment', 'Eyewear', 'eBooks']


In [29]:
# Encoding the product category
le = preprocessing.LabelEncoder()
category_encoded=le.fit_transform(processed_df['product_category_tree'])
processed_df['product_category_tree'] = category_encoded

In [30]:
#Printing the encoded value and their corresponding values
for i in range(28):
    print(str(i) + "-" + le.inverse_transform([i]))

['0-Automotive']
['1-Baby Care']
['2-Bags, Wallets & Belts']
['3-Beauty and Personal Care']
['4-Cameras & Accessories']
['5-Clothing']
['6-Computers']
['7-Eyewear']
['8-Footwear']
['9-Furniture']
['10-Gaming']
['11-Health & Personal Care Appliances']
['12-Home & Kitchen']
['13-Home Decor & Festive Needs']
['14-Home Entertainment']
['15-Home Furnishing']
['16-Home Improvement']
['17-Jewellery']
['18-Kitchen & Dining']
['19-Mobiles & Accessories']
['20-Pens & Stationery']
['21-Pet Supplies']
['22-Sports & Fitness']
['23-Sunglasses']
['24-Tools & Hardware']
['25-Toys & School Supplies']
['26-Watches']
['27-eBooks']


In [31]:
processed_df.head()

,product_category_tree,description
0,5,key features of alisha solid women s cycling s...
1,9,fabhomedecor fabric double sofa bed finish col...
2,8,key features of aw bellies sandals wedges heel...
3,5,key features of alisha solid women s cycling s...
4,21,specifications of sicons all purpose arnica do...


We have taken only those products ehich have a definite categories.<br>
In the code cell below we can notice the change in the shape of our dataframe.

In [32]:
#Shape of processed_df
processed_df.shape

(19661, 2)

In [33]:
'''
This code block is for spliting train test data

'''
X_train, X_test, y_train, y_test = train_test_split(processed_df['description'],processed_df['product_category_tree'],test_size=0.2)

In [34]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15728,)
(3933,)
(15728,)
(3933,)


In [35]:
print(X_train.head())

7907     specifications of o h m solid round neck casua...
6424     buy lava w g router only for rs from flipkart ...
2717     s s stylish women s full coverage bra price rs...
13482    buy dailyobjects back cover for apple ipad onl...
13315    karatcraft purezza yellow gold diamond k ring ...
Name: description, dtype: object


In [36]:
print(y_train.head())

7907      5
6424      6
2717      5
13482    19
13315    17
Name: product_category_tree, dtype: int32


Scikit-learn’s CountVectorizer is used to transform a corpora of text to a vector of term / token counts. It also provides the capability to preprocess your text data prior to generating the vector representation making it a highly flexible feature representation module for text.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html<br>
https://developers.google.com/machine-learning/guides/text-classification/step-3#vectorization

In [37]:
'''
This code block is for converting the training data to vectorized form

'''
# Instantiating the vectorizer and removing stop words
vect = CountVectorizer(analyzer='word', stop_words = 'english')
# Converting the train data
X_train_matrix = vect.fit_transform(X_train) 

In [38]:
# vect.get_feature_names()
X_train_matrix.shape

(15728, 17774)

rows x columns<br>
document = rows<br>
term = columns<br>

In [39]:
vect.vocabulary_

{'specifications': 14671,
 'solid': 14544,
 'round': 13270,
 'neck': 10416,
 'casual': 2408,
 'men': 9731,
 'sweater': 15407,
 'details': 4098,
 'sleeve': 14336,
 'sleeveless': 14338,
 'closure': 2902,
 'zipper': 17722,
 'fastening': 5585,
 'hooded': 7252,
 'yes': 17602,
 'number': 10670,
 'contents': 3301,
 'sales': 13446,
 'package': 11089,
 'pack': 11088,
 'reversible': 13038,
 'fabric': 5462,
 'cotton': 3421,
 'fleece': 5893,
 'weave': 17181,
 'type': 16413,
 'general': 6382,
 'pattern': 11297,
 'occasion': 10737,
 'ideal': 7437,
 'box': 1858,
 'additional': 215,
 'sleek': 14323,
 'style': 15142,
 'code': 2978,
 'fwohmus': 6268,
 'care': 2324,
 'hand': 6854,
 'wash': 17114,
 'rub': 13319,
 'iron': 7920,
 'embellishment': 4925,
 'inside': 7759,
 'dry': 4588,
 'clean': 2836,
 'buy': 2159,
 'lava': 8772,
 'router': 13277,
 'rs': 13306,
 'flipkart': 5910,
 'com': 3056,
 'genuine': 6400,
 'products': 12150,
 'day': 3849,
 'replacement': 12902,
 'guarantee': 6761,
 'free': 6127,
 'shippi

In [40]:
'''
This code block is for training vectorized data

'''
# Instantiate a Multinomial Naive Bayes model
model=MultinomialNB()

# Train the model
model.fit(X_train_matrix, y_train)

# Scoring the trained model
print("Accuracy of model on training data:" , model.score(X_train_matrix, y_train))

Accuracy of model on training data: 0.9491988809766022


In [41]:
'''
This code block is for predicting & scoring test data

'''

# Converting the test data
X_test_matrix = vect.transform(X_test) 

# Scoring for the test data
print ("Accuracy of model on testing data:", model.score(X_test_matrix, y_test))

Accuracy of model on testing data: 0.935163996948894


In [42]:
'''
Time taken by our model to traing itself
'''
%time model.fit(X_train_matrix, y_train)

Wall time: 76.8 ms


MultinomialNB()

In [43]:
# Printing score 
predicted_result=model.predict(X_test_matrix)
print(classification_report(y_test,predicted_result))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       241
           1       0.87      0.67      0.76        98
           2       0.64      0.49      0.55        43
           3       0.83      0.92      0.87       140
           4       1.00      0.40      0.57        10
           5       0.99      0.99      0.99      1236
           6       0.86      0.78      0.81       107
           8       0.98      0.98      0.98       258
           9       1.00      0.98      0.99        41
          10       1.00      0.50      0.67         8
          11       1.00      0.64      0.78        11
          12       1.00      0.29      0.44         7
          13       0.90      0.98      0.94       182
          14       0.50      0.33      0.40         3
          15       0.90      0.99      0.95       140
          16       1.00      0.47      0.64        19
          17       0.89      1.00      0.94       670
          18       0.96    

We can see clearly that Count Vectorizer give number of frequency with respect to index of vocabulary where as tf-idf consider overall documents of weight of words.<br>
Medium article on TfidfVectorizer- https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a<br>
Scikit learn documentation- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [44]:
'''
This code block is for converting the training data to Tf-Idf form

'''
# Removing stop words
vectorizer = TfidfVectorizer(analyzer = 'word' , stop_words = 'english')
# Converting the train data
X_train_tfidf = vectorizer.fit_transform(X_train)

In [45]:
'''
This code block is for training

'''
# Instantiate a Multinomial Naive Bayes model
model2=MultinomialNB()

# Fitting to multinomial NB model
model2.fit(X_train_tfidf, y_train)

# Scoring the trained model (Expected to be above 95 percent)
print("Accuracy of model on training data:" , model2.score(X_train_tfidf, y_train))

Accuracy of model on training data: 0.8835198372329603


In [46]:
'''
This code block is for predicting & scoring test data

'''
# Converting the test data
X_test_tfidf = vectorizer.transform(X_test) 

# Printing score
print ("Accuracy of model on testing data:" , model2.score(X_test_tfidf, y_test))

Accuracy of model on testing data: 0.863208746503941


We used two vectorizer here, CountVectorizer and TfidfVectorizer, and out of these two vectorizer we can observe that the CountVectorizer performed better than TfidfVectorizer.<br>
We will use "model" to predict the product category in the following clode cells

In [47]:
nr_correct = (y_test == model.predict(vect.transform(X_test))).sum()

In [48]:
print("Out of ", X_test.shape ,"target categories, the model predicted", nr_correct, "categories correctly")

Out of  (3933,) target categories, the model predicted 3678 categories correctly


In [49]:
X_test

15632    specifications of sayitloud full sleeve solid ...
5458     key features of tucasa lg table lamp multicolo...
18751    key features of indiweaves regular fit men s l...
17305    thelostpuppy back cover for apple ipad air mul...
13280    twisha bear sitting inch purple price rs made ...
                               ...                        
3579     buy rajesh digital thankyou ceramic mug for rs...
3027     key features of estilo sipper ml sipper ml spe...
13925    orange and orchid striped men s polo t shirt b...
13762    key features of be printed baby boy s round ne...
15868    pet club pc m pet bed multicolor price rs pet ...
Name: description, Length: 3933, dtype: object

In [50]:
test_desc = X_test[13925]
test_desc

'orange and orchid striped men s polo t shirt buy dark blue orange and orchid striped men s polo t shirt for only rs online in india shop online for apparels huge collection of branded clothes only at flipkart com'

In [51]:
'''
Testing Block: Test your string
'''
le.inverse_transform(model2.predict(vect.transform([test_desc])))

array(['Clothing'], dtype=object)